In [ ]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6"
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

sc = spark.sparkContext

In [ ]:
filepath='file:////home/talentum/mishika/source/08_08_2024/*.json'
stock_data=spark.read.option("multiline","true").json(filepath)

from pyspark.sql.functions import explode

stock_data = stock_data.select("symbol", explode("historical").alias("historical"))
stock_data = stock_data.select("symbol", "historical.*")
stock_data.write.mode("overwrite").saveAsTable("STOCKS")

In [ ]:
from pyspark.sql.functions import *

stock_data = spark.sql("select * from stocks")
stock_data = stock_data.withColumn("date",to_date(stock_data.date,"yyyy-MM-dd"))

stock_15 = stock_data.groupby("date",). \
pivot("symbol",["AAPL","MSFT","NVDA","GOOGL","META","AMZN","TSLA","INTC","CRM","ADBE","GM","F","HMC","TM","NSANY", \
                "BMWYY","MBGYY","AMD","TXN","QCOM","AVGO","MU","AMAT","LRCX","JPM","BAC","WFC","GS", \
                "MS","C","AXP","MA","V","XOM","CVX","COP","EOG","NEE","DUK","SO","EXC","PPL","WMB","LMT","RTX", \
                "BA","NOC","GD","HPE","LHX"]).sum("adjClose")

In [ ]:
stock_15 = stock_15.sort("date")
stock_15.show()

+----------+-----+------+----+-----+------+-----+-----+-----+------+----------+-----+----+-----+------+-----+-----+-----+---------+------+-----+-----+-----+-----+------+-----+-----+-----+------+-----+-----+------+------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+-----+----------+------+------+-----+------+
|      date| AAPL|  MSFT|NVDA|GOOGL|  META| AMZN| TSLA| INTC|   CRM|      ADBE|   GM|   F|  HMC|    TM|NSANY|BMWYY|MBGYY|      AMD|   TXN| QCOM| AVGO|   MU| AMAT|  LRCX|  JPM|  BAC|  WFC|    GS|   MS|    C|   AXP|    MA|     V|  XOM|  CVX|  COP|  EOG|  NEE|  DUK|   SO|  EXC|  PPL|  WMB|   LMT|  RTX|        BA|   NOC|    GD|  HPE|   LHX|
+----------+-----+------+----+-----+------+-----+-----+-----+------+----------+-----+----+-----+------+-----+-----+-----+---------+------+-----+-----+-----+-----+------+-----+-----+-----+------+-----+-----+------+------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+-----+----------+------+----

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql import functions as F

return_df=stock_15
my_window = Window.partitionBy().orderBy("date")
for col_name in return_df.columns:
    if col_name!='date':
     return_df = return_df.withColumn(col_name, F.when(F.isnull(col(col_name) - F.lag(col(col_name)).over(my_window)), 0)
                                      .otherwise(col(col_name) - F.lag(col(col_name)).over(my_window))/F.lag(col(col_name)).over(my_window))
for c in return_df.columns:
    if c!='date':
        return_df = return_df.withColumn(c, F.round(c, 6))

return_df=return_df.dropna()
return_df=return_df.drop("date")
return_df.show(5)

+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|     AAPL|     MSFT|     NVDA|    GOOGL|     META|     AMZN|     TSLA|     INTC|      CRM|     ADBE|       GM|        F|      HMC|       TM|    NSANY|    BMWYY|    MBGYY|      AMD|      TXN|     QCOM|     AVGO|       MU|     AMAT|     LRCX|      JPM|      BAC|      WFC|       GS|       MS|        C|      AXP|       MA|        V|      XOM|      CVX|      COP|      EOG|      NEE|      DUK|       SO|      EXC|      PPL|      WMB|      LMT|      RTX|       BA|      NOC|       GD|      HPE|      L

In [ ]:
from pyspark.mllib.stat import Statistics
import pandas as pd
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql.window import Window
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation

In [ ]:
def stocks_correlation(stocks_df,cols):
    # convert to vector column first
    vector_col = "corr_features"
    assembler = VectorAssembler(inputCols=cols, outputCol=vector_col)
    stocks_df_vector = assembler.transform(stocks_df).select(vector_col)

    # get correlation matrix
    matrix = Correlation.corr(stocks_df_vector, vector_col).collect()[0][0]
    corrmatrix = matrix.toArray().tolist()

    df = spark.createDataFrame(corrmatrix,cols)

    for c in df.columns:
        df = df.withColumn(c,F.round(c,2))

    stock_list = df.columns #["AAPL","KO","NFLX","DIS","IBM","VZ","WMT","GE","TSLA","MA","AMZN","MSFT","UN","V"]

    # Create a DataFrame from the list of values
    values_df = spark.createDataFrame([Row(new_column=v) for v in stock_list])

    df = df.withColumn("row_index", monotonically_increasing_id())
    values_df = values_df.withColumn("row_index", monotonically_increasing_id())

    # Join the DataFrames on the row index
    corr_df = df.join(values_df, on="row_index").drop("row_index")

    # Show the resulting DataFrame
    corr_df=corr_df.withColumnRenamed("new_column","stockName")
    return corr_df

In [ ]:
corr_final_df = stocks_correlation(return_df,return_df.columns)
corr_final_df.show()

+----+----+----+-----+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---------+
|AAPL|MSFT|NVDA|GOOGL|META|AMZN|TSLA|INTC| CRM|ADBE|  GM|   F| HMC|  TM|NSANY|BMWYY|MBGYY| AMD| TXN|QCOM|AVGO|  MU|AMAT|LRCX| JPM| BAC| WFC|  GS|  MS|   C| AXP|  MA|   V| XOM| CVX| COP| EOG| NEE| DUK|  SO| EXC| PPL| WMB| LMT| RTX|  BA| NOC|  GD| HPE| LHX|stockName|
+----+----+----+-----+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---------+
|0.38|0.38| 0.3| 0.38|0.28|0.25|0.25|0.39|0.32|0.29| 0.6|0.55|0.51|0.45| 0.35| 0.55|  0.6|0.29|0.48|0.38|0.39|0.45|0.44|0.43|0.82|0.86| 1.0|0.77|0.77|0.82|0.73|0.54|0.54|0.57|0.58|0.55|0.51|0.34|0.44|0.

In [ ]:
def moderateCorrelation(stock_df,stock_list):
    st_lst = stock_list
    portfolios = []
    for st in st_lst:
        protfolio_st = stock_df[[st,"stockName"]]
        stock_df.createOrReplaceTempView("aapstock")
        query = "select stockName from aapstock where {} between 0.5 and 0.69".format(st)
        portfolio_stocks = spark.sql(query)
        portfolio_i = portfolio_stocks.select("stockName").rdd.flatMap(lambda x:x).collect()
        portfolio_i.append(st)
        portfolios.append(portfolio_i)
    return portfolios

In [ ]:
def highCorrelation(stock_df,stock_list):
    st_lst = stock_list
    portfolios = []
    for st in st_lst:
        protfolio_st = stock_df[[st,"stockName"]]
        stock_df.createOrReplaceTempView("aapstock")
        query = "select stockName from aapstock where {} between 0.7 and 0.99".format(st)
        portfolio_stocks = spark.sql(query)
        portfolio_i = portfolio_stocks.select("stockName").rdd.flatMap(lambda x:x).collect()
        portfolio_i.append(st)
        portfolios.append(portfolio_i)
    return portfolios

In [ ]:
def lowCorrelation(stock_df,stock_list):
    st_lst = stock_list
    portfolios = []
    for st in st_lst:
        protfolio_st = stock_df[[st,"stockName"]]
        stock_df.createOrReplaceTempView("aapstock")
        query = "select stockName from aapstock where {} between 0.3 and 0.49".format(st)
        portfolio_stocks = spark.sql(query)
        portfolio_i = portfolio_stocks.select("stockName").rdd.flatMap(lambda x:x).collect()
        portfolio_i.append(st)
        portfolios.append(portfolio_i)
    return portfolios

In [ ]:
stock_lst = ["AAPL","MSFT","TSLA","BMWYY","QCOM","AVGO","JPM","BAC","XOM","CVX","LMT","RTX"]

In [ ]:
moderated_correlation_portfolios = moderateCorrelation(corr_final_df,stock_lst)


['QCOM', 'AMAT', 'INTC', 'V', 'MA', 'ADBE', 'AMD', 'MS', 'AMZN', 'GOOGL', 'CRM', 'NVDA', 'META', 'TXN', 'MU', 'LRCX', 'AVGO', 'AAPL']
['QCOM', 'AMAT', 'INTC', 'V', 'MA', 'AMD', 'MS', 'AMZN', 'CRM', 'NVDA', 'META', 'TXN', 'MU', 'LRCX', 'AVGO', 'MSFT']
['TSLA']
['WFC', 'C', 'CVX', 'V', 'MA', 'BAC', 'GS', 'MS', 'GM', 'HPE', 'HMC', 'F', 'AXP', 'BA', 'JPM', 'BMWYY']
['AAPL', 'INTC', 'V', 'MA', 'ADBE', 'AMD', 'AMZN', 'MSFT', 'GOOGL', 'NVDA', 'MU', 'QCOM']
['AAPL', 'INTC', 'V', 'MA', 'ADBE', 'GS', 'AMD', 'MS', 'AMZN', 'MSFT', 'HPE', 'GOOGL', 'CRM', 'MU', 'MBGYY', 'AVGO']
['CVX', 'V', 'MA', 'PPL', 'XOM', 'GM', 'HPE', 'RTX', 'HMC', 'F', 'COP', 'EOG', 'TXN', 'BMWYY', 'WMB', 'EXC', 'MBGYY', 'BA', 'GD', 'JPM']
['CVX', 'V', 'MA', 'PPL', 'XOM', 'GM', 'HPE', 'RTX', 'HMC', 'F', 'COP', 'EOG', 'TXN', 'BMWYY', 'WMB', 'EXC', 'MBGYY', 'BA', 'GD', 'BAC']
['WFC', 'C', 'BAC', 'GS', 'MS', 'RTX', 'AXP', 'WMB', 'MBGYY', 'GD', 'JPM', 'XOM']
['WFC', 'C', 'V', 'MA', 'BAC', 'GS', 'PPL', 'MS', 'GM', 'HPE', 'RTX', 'BM

In [ ]:
high_correlation_portfolios = highCorrelation(corr_final_df,stock_lst)

['MSFT', 'AAPL']
['AAPL', 'ADBE', 'GOOGL', 'MSFT']
['TSLA']
['MBGYY', 'BMWYY']
['AMAT', 'TXN', 'LRCX', 'AVGO', 'QCOM']
['QCOM', 'AMAT', 'NVDA', 'TXN', 'LRCX', 'AVGO']
['WFC', 'C', 'BAC', 'GS', 'MS', 'AXP', 'JPM']
['WFC', 'C', 'GS', 'MS', 'AXP', 'JPM', 'BAC']
['CVX', 'COP', 'EOG', 'XOM']
['XOM', 'COP', 'EOG', 'WMB', 'CVX']
['LHX', 'NOC', 'GD', 'LMT']
['GD', 'RTX']


In [ ]:
low_correlation_portfolios = lowCorrelation(corr_final_df,stock_lst)

['WFC', 'C', 'CVX', 'LMT', 'SO', 'BAC', 'TSLA', 'GS', 'PPL', 'XOM', 'GM', 'HPE', 'RTX', 'NEE', 'HMC', 'F', 'LHX', 'COP', 'TM', 'NSANY', 'BMWYY', 'DUK', 'AXP', 'WMB', 'EXC', 'MBGYY', 'BA', 'GD', 'JPM', 'AAPL']
['WFC', 'C', 'CVX', 'LMT', 'SO', 'BAC', 'TSLA', 'GS', 'PPL', 'GM', 'HPE', 'RTX', 'NEE', 'HMC', 'F', 'LHX', 'TM', 'NSANY', 'BMWYY', 'DUK', 'AXP', 'WMB', 'EXC', 'MBGYY', 'BA', 'GD', 'JPM', 'MSFT']
['C', 'QCOM', 'AAPL', 'AMAT', 'INTC', 'V', 'MA', 'ADBE', 'GS', 'AMD', 'MS', 'AMZN', 'MSFT', 'GM', 'GOOGL', 'HMC', 'CRM', 'F', 'NVDA', 'META', 'TXN', 'MU', 'BMWYY', 'AXP', 'LRCX', 'AVGO', 'MBGYY', 'BA', 'TSLA']
['QCOM', 'AAPL', 'AMAT', 'INTC', 'LMT', 'SO', 'TSLA', 'ADBE', 'AMD', 'PPL', 'XOM', 'MSFT', 'RTX', 'GOOGL', 'NEE', 'CRM', 'LHX', 'COP', 'NVDA', 'META', 'TM', 'EOG', 'TXN', 'NSANY', 'MU', 'DUK', 'WMB', 'LRCX', 'AVGO', 'EXC', 'GD', 'BMWYY']
['WFC', 'C', 'CVX', 'BAC', 'TSLA', 'GS', 'PPL', 'MS', 'GM', 'HPE', 'RTX', 'NEE', 'HMC', 'CRM', 'F', 'META', 'TM', 'NSANY', 'BMWYY', 'AXP', 'WMB', 'E

In [ ]:
lst = ['AAPL', 'ADBE', 'GOOGL', 'MSFT']
MSFT_high_corr = stocks_correlation(return_df,lst)
MSFT_high_corr.show()

+----+----+-----+----+---------+
|AAPL|ADBE|GOOGL|MSFT|stockName|
+----+----+-----+----+---------+
| 1.0|0.63| 0.66|0.75|     AAPL|
|0.63| 1.0| 0.65|0.75|     ADBE|
|0.75|0.75| 0.76| 1.0|     MSFT|
|0.66|0.65|  1.0|0.76|    GOOGL|
+----+----+-----+----+---------+



In [ ]:
lst = ['QCOM', 'AMAT', 'INTC', 'V', 'MA', 'AMD', 'MS', 'AMZN', 'CRM', 'NVDA', 'META', 'TXN', 'MU', 'LRCX', 'AVGO', 'MSFT']
MSFT_modreated_corr = stocks_correlation(return_df,lst)
MSFT_modreated_corr.show()

+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---------+
|QCOM|AMAT|INTC|   V|  MA| AMD|  MS|AMZN| CRM|NVDA|META| TXN|  MU|LRCX|AVGO|MSFT|stockName|
+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---------+
| 1.0|0.74|0.58| 0.5| 0.5|0.62|0.48|0.52|0.49|0.66|0.49|0.73|0.66|0.72|0.72|0.62|     QCOM|
|0.52|0.51|0.43|0.42|0.43|0.56|0.35| 1.0|0.56|0.59|0.61|0.51|0.45| 0.5| 0.5|0.68|     AMZN|
|0.48|0.54|0.47|0.62|0.63|0.39| 1.0|0.35| 0.4|0.44|0.37|0.58|0.52|0.54| 0.5|0.52|       MS|
|0.66|0.71|0.49| 0.5| 0.5|0.75|0.44|0.59|0.55| 1.0|0.54|0.64|0.64| 0.7| 0.7|0.69|     NVDA|
|0.62|0.66|0.48|0.45|0.45| 1.0|0.39|0.56| 0.5|0.75|0.49|0.61|0.58|0.67|0.61|0.61|      AMD|
|0.74| 1.0|0.62|0.54|0.56|0.66|0.54|0.51| 0.5|0.71| 0.5|0.77|0.75|0.93|0.75|0.65|     AMAT|
|0.49| 0.5|0.41|0.45|0.46|0.49|0.37|0.61|0.52|0.54| 1.0|0.49|0.46|0.49|0.49|0.62|     META|
| 0.5|0.54|0.44| 1.0|0.91|0.45|0.62|0.42|0.51| 0.5|0.45|0.58| 0.5|0.55|0.54|0.63

In [ ]:

lst =['WFC', 'C', 'CVX', 'LMT', 'SO', 'BAC', 'TSLA', 'GS', 'PPL', 'GM', 'HPE', 'RTX', 'NEE', 'HMC', 'F', 'LHX', 'TM', 'NSANY', 'BMWYY', 'DUK', 'AXP', 'WMB', 'EXC', 'MBGYY', 'BA', 'GD', 'JPM', 'MSFT']
MSFT_low_corr = stocks_correlation(return_df,lst)
MSFT_low_corr.show()

+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+-----+----+----+----+----+-----+----+----+----+----+---------+
| WFC|   C| CVX| LMT|  SO| BAC|TSLA|  GS| PPL|  GM| HPE| RTX| NEE| HMC|   F| LHX|  TM|NSANY|BMWYY| DUK| AXP| WMB| EXC|MBGYY|  BA|  GD| JPM|MSFT|stockName|
+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+-----+----+----+----+----+-----+----+----+----+----+---------+
|0.82|0.86|0.64|0.42|0.45| 0.9|0.27|0.83|0.57|0.63|0.59|0.63|0.37|0.55|0.57|0.44|0.47| 0.38| 0.58|0.47|0.78|0.59|0.51| 0.63|0.59|0.63| 1.0|0.43|      JPM|
|0.44|0.43|0.42|0.53|0.86|0.46|0.12|0.44|0.78|0.33|0.31|0.48| 0.7|0.35|0.35|0.53|0.32| 0.23| 0.33| 1.0|0.45|0.37|0.79|  0.4|0.36|0.57|0.47|0.36|      DUK|
| 1.0|0.82|0.58|0.41|0.43|0.86|0.25|0.77|0.53| 0.6|0.58|0.59|0.34|0.51|0.55|0.45|0.45| 0.35| 0.55|0.44|0.73|0.49| 0.5|  0.6|0.57|0.59|0.82|0.38|      WFC|
| 0.5|0.48|0.47|0.51| 0.8| 0.5|0.22|0.48|0.76| 0.4|0.38|0.53|0.67|0.43